#### Import necessary packages


In [41]:
# Standard library imports
import cupy as cp
import numpy as np
# from numpy.lib.stride_tricks import sliding_window_view
from cupy.lib.stride_tricks import sliding_window_view
from typing import Tuple

# Progress bar
from tqdm.auto import tqdm

# Third-party imports
import torch
from torch import nn
from torch.utils.data import DataLoader
from einops import einsum, rearrange
from torchvision.datasets import ImageNet
from torchvision.models import AlexNet_Weights

#### Load the weights and biases of AlexNet


In [42]:
weights_and_biases = AlexNet_Weights.DEFAULT.get_state_dict()
print(weights_and_biases.keys())

odict_keys(['features.0.weight', 'features.0.bias', 'features.3.weight', 'features.3.bias', 'features.6.weight', 'features.6.bias', 'features.8.weight', 'features.8.bias', 'features.10.weight', 'features.10.bias', 'classifier.1.weight', 'classifier.1.bias', 'classifier.4.weight', 'classifier.4.bias', 'classifier.6.weight', 'classifier.6.bias'])


#### Load the data


In [ ]:
def default_collate(batch):
    """
    A collation function that simply returns the batch as is.
    We convert torch tensors to numpy arrays since np.pad doesn't work on tensors
    """
    imgs, labels = zip(*batch)  # imgs: tuple[torch.Tensor], labels: tuple[int]
    imgs = [cp.asarray(img.numpy(), dtype=cp.float32) for img in imgs]
    imgs = cp.stack(imgs, axis=0)                      # [B,3,224,224]
    labels = cp.asarray(labels, dtype=cp.int64)        # [B]
    return imgs, labels


# implement using ImageNet
imagenet_val = ImageNet(
    root="data/ImageNet1k",
    split="val",
    transform=AlexNet_Weights.IMAGENET1K_V1.transforms()
)

# the dataloader automatically segregates the labels
val_dataloader = DataLoader(
    imagenet_val,
    batch_size=512,
    shuffle=False,
    num_workers=0,
    collate_fn=default_collate
)

#### Define the custom Conv2d

Here we just inherit from nn.Module so that it will work cleanly with pytorch even though the implementation of the computation is done by einops, einsum, and numpy.


In [ ]:
class PatchMixin:
    def __init__(self, kernel_size: int, stride: int) -> None:
        super().__init__()
        self.kernel_size = kernel_size
        self.stride = stride

    def _patch_with_stride(self, x_pad: cp.ndarray) -> cp.ndarray:
        """
        Extract k x k patches (kernel dims) from the input array with the given stride.

        Args:
            x_pad (cp.ndarray): Input array of shape (b, c, h, w).

        Returns:
            cp.ndarray: Array of shape (b, c, h/stride, w/stride, k, k) containing the extracted patches.
        """
        windows = sliding_window_view(  # type: ignore
            x_pad,
            window_shape=(self.kernel_size, self.kernel_size),
            axis=(-2, -1)  # type: ignore
        )
        return windows[:, :, ::self.stride, ::self.stride, :, :]


class WeightsAndBiasMixin:
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

    def init_weights_and_bias(self, weight_loc: str, bias_loc: str) -> Tuple[cp.ndarray, cp.ndarray]:
        weight_np = weights_and_biases[weight_loc].detach().cpu().numpy()
        bias_np = weights_and_biases[bias_loc].detach().cpu().numpy()
        weight = cp.asarray(weight_np)
        bias = cp.asarray(bias_np)
        return weight, bias


class CustomConv2d(WeightsAndBiasMixin, PatchMixin, nn.Module):
    """
    2D Convolution layer using CuPy, Einops, and einsum (einsum from einops handles cupy).
    Limited shape flexibility for demonstration/inference.
    """

    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        kernel_size: int,
        stride: int = 1,
        padding: int = 0,
        weight_loc: str = '',
        bias_loc: str = '',
    ) -> None:
        super().__init__(kernel_size, stride)
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding

        self.weight, self.bias = self.init_weights_and_bias(
            weight_loc, bias_loc)
        self.reset_parameters()

    def reset_parameters(self) -> None:
        # Using pretrained weights only; no init required for now.
        pass

    def _apply_padding(self, x: cp.ndarray) -> cp.ndarray:
        if self.padding == 0:
            return x
        return cp.pad(
            x,
            pad_width=((0, 0), (0, 0), (self.padding, self.padding),
                       (self.padding, self.padding)),
            mode='constant',
            constant_values=0,
        )

    def forward(self, x: cp.ndarray) -> cp.ndarray:
        x_pad = self._apply_padding(x)
        patched_windows = self._patch_with_stride(x_pad)
        pre_activation = einsum(
            patched_windows, self.weight, 'b c w h kw kh, o c kw kh -> b o w h')
        return pre_activation + self.bias[None, :, None, None]  # type: ignore

In [45]:
class CustomReLU(nn.Module):
    """Custom ReLU activation (CuPy)."""

    def forward(self, x: cp.ndarray) -> cp.ndarray:
        return cp.maximum(x, 0.0)

In [46]:
class CustomMaxPool2d(PatchMixin, nn.Module):
    """Custom Max Pooling layer (CuPy)."""

    def __init__(self, kernel_size: int, stride: int) -> None:
        super().__init__(kernel_size, stride)

    def forward(self, x: cp.ndarray) -> cp.ndarray:
        patched_windows = self._patch_with_stride(x)
        return cp.max(patched_windows, axis=(-2, -1))

In [47]:
class CustomAdaptiveAvgPool2d(nn.Module):
    """Custom Adaptive Average Pooling layer (CuPy, inference only)."""

    def __init__(self, output_size: tuple[int, int]) -> None:
        super().__init__()
        if isinstance(output_size, int):
            self.output_size = (output_size, output_size)
        else:
            self.output_size = output_size

    def forward(self, x: cp.ndarray) -> cp.ndarray:
        b, c, h, w = x.shape
        out_h, out_w = self.output_size
        out = cp.zeros((b, c, out_h, out_w), dtype=x.dtype)
        for i in range(out_h):
            h_start = int(cp.floor(i * h / out_h))
            h_end = int(cp.ceil((i + 1) * h / out_h))
            for j in range(out_w):
                w_start = int(cp.floor(j * w / out_w))
                w_end = int(cp.ceil((j + 1) * w / out_w))
                region = x[:, :, h_start:h_end, w_start:w_end]
                out[:, :, i, j] = region.mean(axis=(-2, -1))
        return out

In [48]:
class EinopsLinear(WeightsAndBiasMixin, nn.Module):
    def __init__(self, in_features: int, out_features: int, weight_loc: str, bias_loc: str):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight, self.bias = self.init_weights_and_bias(
            weight_loc, bias_loc)

    def forward(self, x: cp.ndarray) -> cp.ndarray:
        y = einsum(x, self.weight, "b i, o i -> b o")
        if self.bias is not None:
            y = y + self.bias
        return y

In [49]:
class AlexNet(nn.Module):
    def __init__(self, num_classes: int = 1000) -> None:
        super().__init__()
        self.features = nn.Sequential(
            CustomConv2d(3, 64, kernel_size=11, stride=4, padding=2,
                         weight_loc='features.0.weight', bias_loc='features.0.bias'),
            CustomReLU(),
            CustomMaxPool2d(kernel_size=3, stride=2),
            CustomConv2d(64, 192, kernel_size=5, padding=2,
                         weight_loc='features.3.weight', bias_loc='features.3.bias'),
            CustomReLU(),
            CustomMaxPool2d(kernel_size=3, stride=2),
            CustomConv2d(192, 384, kernel_size=3, padding=1,
                         weight_loc='features.6.weight', bias_loc='features.6.bias'),
            CustomReLU(),
            CustomConv2d(384, 256, kernel_size=3, padding=1,
                         weight_loc='features.8.weight', bias_loc='features.8.bias'),
            CustomReLU(),
            CustomConv2d(256, 256, kernel_size=3, padding=1,
                         weight_loc='features.10.weight', bias_loc='features.10.bias'),
            CustomReLU(),
            CustomMaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = CustomAdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            EinopsLinear(256 * 6 * 6, 4096, weight_loc='classifier.1.weight',
                         bias_loc='classifier.1.bias'),
            CustomReLU(),
            EinopsLinear(4096, 4096, weight_loc='classifier.4.weight',
                         bias_loc='classifier.4.bias'),
            CustomReLU(),
            EinopsLinear(4096, num_classes, weight_loc='classifier.6.weight',
                         bias_loc='classifier.6.bias'),
        )

    def forward(self, x: cp.ndarray) -> cp.ndarray:
        x = self.features(x)
        x = self.avgpool(x)
        b = x.shape[0]
        x = x.reshape(b, -1)  # flatten
        x = self.classifier(x)
        return x

In [50]:
model = AlexNet()
model.eval()

total = 0
correct = 0

# Estimate total batches for progress bar length
try:
    total_batches = len(val_dataloader)
except TypeError:
    total_batches = None

for images, labels in tqdm(val_dataloader, total=total_batches, desc="Evaluating", leave=True):
    # ensure CuPy arrays
    if isinstance(images, torch.Tensor):
        images = cp.asarray(images.numpy())
    if isinstance(labels, torch.Tensor):
        labels = cp.asarray(labels.numpy())

    outputs = model.forward(images)  # (b, num_classes) cp.ndarray
    predicted = cp.argmax(outputs, axis=1)
    total += int(labels.shape[0])
    correct += int((predicted == labels).sum())
    running_acc = correct / total if total else 0.0
    tqdm.write(f"Running Acc: {running_acc:.4f}")

accuracy = correct / total if total else 0.0
print(f"Validation Accuracy: {accuracy:.4f}")

Evaluating:   1%|          | 1/98 [00:01<02:40,  1.65s/it]

Running Acc: 0.7109


Evaluating:   2%|▏         | 2/98 [00:02<02:18,  1.45s/it]

Running Acc: 0.7500


Evaluating:   3%|▎         | 3/98 [00:04<02:14,  1.42s/it]

...

Running Acc: 0.5628


Evaluating:  99%|█████████▉| 97/98 [02:19<00:01,  1.46s/it]

Running Acc: 0.5653


Evaluating: 100%|██████████| 98/98 [02:20<00:00,  1.43s/it]

Running Acc: 0.5656
Validation Accuracy: 0.5656
